In [8]:
# Parameters
n_cross_val = 6
fraction_training = 1.0
data_file = "../../data/experiments/pilot_random1_player_round_slim.csv"
data_dir = "../../data/artificial_humans/dev"
labels = {}
model_name = "graph"
job_id = 'dev'
model_args = {
    "y_levels": 31,
    "y_name": "punishments",
    "add_rnn": False,
    "add_edge_model": False,
    "add_global_model": False,
    "hidden_size": 10,
    "x_encoding": [
        {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
        {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
        {"name": "prev_valid", "etype": "bool"},
        {"name": "episode_group_idx", "n_levels": 86, "encoding": "projection"},
    ],
    "u_encoding": [
        {"name": "prev_common_good", "norm": 32, "etype": "float"},
        {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
    ],
}
mask_name = "manager_valid"
experiment_names = ["trail_rounds_2"]
optimizer_args = {"lr": 0.001, "weight_decay": 1e-05}
train_args = {"epochs": 100, "batch_size": 20, "clamp_grad": 1, "eval_period": 10, 'l1_entropy': 0.1}
n_player = 4
shuffle_features = ["prev_punishments", "prev_contributions", "prev_common_good"]
device = "cpu"
seed = 123


In [9]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import torch as th
from aimanager.generic.data import create_torch_data, get_cross_validations
from aimanager.artificial_humans import AH_MODELS
from aimanager.artificial_humans.evaluation import Evaluator
from aimanager.utils.array_to_df import using_multiindex
from aimanager.utils.utils import make_dir
from aimanager.generic.graph_encode import create_fully_connected
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader

model_dir = os.path.join(data_dir, 'model')
make_dir(model_dir)

th.random.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_csv(data_file)

df = df[df['experiment_name'].isin(experiment_names)]

data, default_values = create_torch_data(df)

In [11]:
data.keys()

dict_keys(['punishments', 'contributions', 'valid', 'recorded', 'common_good', 'round_number', 'is_first', 'manager_valid', 'episode_group_idx', 'prev_punishments', 'prev_contributions', 'prev_valid', 'prev_recorded', 'prev_common_good', 'prev_manager_valid'])

In [12]:
th_device = th.device(device)

metrics = []
confusion_matrix = []
syn_pred = []
ev = Evaluator()

th_device = th.device(device)

syn_index = ['prev_punishments', 'prev_contributions']
edge_index = create_fully_connected(n_player)


def shuffle_feature(data, feature_name):
    data = {**data}
    data[feature_name] = data[feature_name][th.randperm(len(data[feature_name]))]
    return data

for i, train_data, test_data in get_cross_validations(data, n_cross_val, fraction_training):
    model = AH_MODELS[model_name](default_values=default_values, **model_args).to(th_device)

    train_data_ = model.encode(train_data, mask=mask_name, edge_index=edge_index)
    if test_data is not None:
        test_data_ = model.encode(test_data, mask=mask_name, edge_index=edge_index)

    optimizer = th.optim.Adam(model.parameters(), **optimizer_args)
    loss_fn = th.nn.CrossEntropyLoss(reduction='none')
    sum_loss = 0
    n_steps = 0

    for e in range(train_args['epochs']):
        ev.set_labels(cv_split=i, epoch=e)
        model.train()
        for j, batch_data in enumerate(iter(DataLoader(train_data_, shuffle=True, batch_size=train_args['batch_size']))):
            optimizer.zero_grad()
            py = model(batch_data).flatten(end_dim=-2)
            y_true = batch_data['y_enc'].flatten(end_dim=-2)
            py_soft = py.softmax(-1)
            loss = loss_fn(py, y_true) + (py_soft * py_soft.log()).sum(-1) * train_args['l1_entropy']

            mask = batch_data['mask'].flatten()
            loss = (loss * mask).sum() / mask.sum()

            loss.backward(retain_graph=True)

            if train_args['clamp_grad']:
                for param in model.parameters():
                    param.grad.data.clamp_(-train_args['clamp_grad'], train_args['clamp_grad'])
            optimizer.step()
            sum_loss += loss.item()
            n_steps +=1
        
        last_epoch = e == (train_args['epochs'] - 1)

        if (e % train_args['eval_period'] == 0) or last_epoch:
            avg_loss = sum_loss/n_steps
            print(f'CV {i} | Epoch {e} | Loss {avg_loss}')
            ev.add_loss(avg_loss)

            ev.eval_set(model, train_data_, calc_confusion=last_epoch, set='train')
            if test_data is not None:
                ev.eval_set(model, test_data_, calc_confusion=last_epoch, set='test')
                for sf in shuffle_features:
                    shuffled_data = shuffle_feature(test_data, sf)
                    shuffled_data = model.encode(shuffled_data, mask=mask_name, edge_index=edge_index)
                    ev.eval_set(model, shuffled_data, calc_confusion=False, set='test', shuffle_feature=sf)
            sum_loss = 0
            n_steps = 0

    if i is None:
        ev.save(data_dir, labels, job_id=job_id)
        model_path = os.path.join(model_dir, f'{job_id}.pt')
        model.save(model_path)

CV 0 | Epoch 0 | Loss 3.136858880519867
CV 0 | Epoch 10 | Loss 2.9536250829696655
CV 0 | Epoch 20 | Loss 2.6307996571063996
CV 0 | Epoch 30 | Loss 2.290650945901871
CV 0 | Epoch 40 | Loss 1.9623196691274643
CV 0 | Epoch 50 | Loss 1.7238907724618913
CV 0 | Epoch 60 | Loss 1.6138409674167633
CV 0 | Epoch 70 | Loss 1.5523815274238586
CV 0 | Epoch 80 | Loss 1.528792029619217
CV 0 | Epoch 90 | Loss 1.5101826429367065
CV 0 | Epoch 99 | Loss 1.5039404067728255
CV 1 | Epoch 0 | Loss 3.2329419255256653
CV 1 | Epoch 10 | Loss 3.014157700538635
CV 1 | Epoch 20 | Loss 2.61902112364769
CV 1 | Epoch 30 | Loss 2.233032748103142
CV 1 | Epoch 40 | Loss 1.9188682585954666
CV 1 | Epoch 50 | Loss 1.7063937157392501
CV 1 | Epoch 60 | Loss 1.598928913474083
CV 1 | Epoch 70 | Loss 1.5426010966300965
CV 1 | Epoch 80 | Loss 1.5247330278158189
CV 1 | Epoch 90 | Loss 1.5000739455223084
CV 1 | Epoch 99 | Loss 1.4980129963821835
CV 2 | Epoch 0 | Loss 2.775436520576477
CV 2 | Epoch 10 | Loss 2.5929281532764437
CV 2